In [1]:

import numpy as np
from qulacs import QuantumCircuit
from itertools import combinations
from mcr.equiv_check import (
    equivalence_check_via_mqt_qcec,
    equiv,
    pauli_bit_equivalence_check,
)
from mcr.gate_apply import (
    PauliBit,
    grouping,
    loop_optimization,
    set_clifford_to_qulacs,
    zhang_optimization,
    multiply_all,
)
import pickle

from mcr.mcr_optimize import find_nontrivial_swap, find_mcr
from opt_using_mcr import test_algorithm, mcr_swap, three_layer_nontrivial_swap

In [2]:
nqubits = 2
with open(f"unopt_{nqubits}.pickle", "rb") as f:
    seq = pickle.load(f)
data = []
for elem in seq:
    sgn = str(elem[1])[0]
    pauli_str = str(elem[1])[1:]
    if sgn == "+":
        data.append(PauliBit(pauli_str, np.pi / 4))
    else:
        assert sgn == "-", f"Unexpected sign: {sgn}"
        data.append(PauliBit(pauli_str, -np.pi / 4))
data.append(PauliBit("Z" * nqubits, -np.pi / 4))  # Add identity gate

In [3]:
seq_a = [PauliBit(ele.get_pauli_str(), ele.get_angle()) for ele in data]
clifford_lst, optimized_data = test_algorithm(seq_a, show_opt_log=True)

🎉 Successful optimization using MCR! 48 -> 26
🎉 Successful optimization using MCR! 26 -> 12
🔍 No optimization found in 3th iteration. Try 100 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 99 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 98 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 97 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 96 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 95 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 94 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 93 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 92 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 91 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 90 times left... 12 -> 12
🔍 No optimization found in 3th iteration. Try 89 times left... 12 -> 12
🔍 No optimization found in 3th iteration. T

何回かループさせないと消せないサンプルが存在する(concatenationと関係？)

In [32]:
tmp = grouping(optimized_data)

In [34]:
from itertools import permutations
from tqdm import tqdm
i = 0
results = []
for perm in tqdm(permutations(tmp)):
    target = sum(perm,[])
    cli, tmp2 = test_algorithm(target, show_opt_log=False)
    if equiv([cli, tmp2], [[], optimized_data]) and len(tmp2) < len(target):
        print("New length:", len(tmp2))
        results.append(tmp2)
    # if len(tmp2) < len(tmp):
    #     tmp = tmp2

7494it [05:22, 23.21it/s]


KeyboardInterrupt: 

In [8]:
equiv([[],seq_a],[clifford_lst, optimized_data])

True

In [9]:
from mcr.gate_apply import set_clifford_to_qulacs
from qulacs.circuit import QuantumCircuitOptimizer as QCO
from qulacs import QuantumCircuit
circ = QuantumCircuit(nqubits)
circ = set_clifford_to_qulacs(circ, clifford_lst)
# for elem in data:
    # circ.merge_circuit(elem.convert_into_qulacs())
QCO().optimize(circ,nqubits)
circ.get_gate(0).get_matrix()

array([[-0.5+0.5j,  0.5-0.5j,  0. +0.j ,  0. +0.j ],
       [-0.5-0.5j, -0.5-0.5j,  0. +0.j ,  0. +0.j ],
       [ 0. +0.j ,  0. +0.j , -0.5-0.5j, -0.5-0.5j],
       [ 0. +0.j ,  0. +0.j ,  0.5-0.5j, -0.5+0.5j]])